In [63]:
from bs4 import BeautifulSoup # For HTML parsing
import urllib # Website connections
import re # Regular expressions
from time import sleep # To prevent overwhelming the server between connections
import nltk
nltk.download('stopwords')
from collections import Counter # Keep track of our term counts
from nltk.corpus import stopwords # Filter out stopwords, such as 'the', 'or', 'and'
import pandas as pd # For converting results to a dataframe and bar chart plots
import csv
%matplotlib inline


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mareks/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [131]:
def skills_info(city = None, state = None):
    
# CSV SETUP:
    
    file = open('indeedData.csv', 'w')
    writer = csv.writer(file)
    writer.writerow(['Title', 'Company', 'Location', 'Summary', 'Link', 'Details', 'Date Posted'])

# Create URL for Beautiful Soup:

    final_job = 'internship' # searching for internship
    
    if city is not None:
        final_city = city.split() 
        final_city = '+'.join(word for word in final_city)
        final_site_list = ['http://www.indeed.com/jobs?q=%22', final_job, '%22&l=', final_city,
                    '%2C+', state] # Join all of our strings together so that indeed will search correctly
    else:
        final_site_list = ['http://www.indeed.com/jobs?q="', final_job, '"']
    final_site = ''.join(final_site_list) # Merge the html address together into one string
    print("Finale site :", final_site) # Print of URL

# Open page with beautiful soup and extract HTML:

    try:
        html = urllib.request.urlopen(final_site).read() # Open up the front page of our search first
    except:
        'That city/state combination did not have any jobs. Exiting . . .' # In case the city is invalid
        return "Not found"

    soup = BeautifulSoup(html) # Get the html from the first page
    jobs = soup.find('td', attrs={'id': 'resultsCol'}).find_all('div', attrs={'class': "jobsearch-SerpJobCard"}, recursive=False)

# Extract the total number of jobs found to calculate number of pages:

    num_jobs_area = soup.find(id = 'searchCount').encode('utf-8') 
    job_numbers = re.findall('\d+', num_jobs_area.decode('utf-8')) # Extract the total jobs found from the search result
    if len(job_numbers) > 4: # Have a total number of jobs greater than 1000
        total_num_jobs = (int(job_numbers[2])) + int(job_numbers[3])
    else:
        total_num_jobs = int(job_numbers[1]) 
    
    city_title = city
    if city is None:
        city_title = 'Nationwide'
        
    print('There were', total_num_jobs, 'jobs found,', city_title) # Display how many jobs were found
    num_pages = total_num_jobs //15 # This will be how we know the number of times we need to iterate over each new
    base_url = 'http://www.indeed.com'
    job_descriptions = [] # Store all our descriptions in this list

# Iterate through pages:

    for i in range(1,num_pages+1): # Loop through all of our search result pages
        print('Getting page', i)
        start_num = str(i*10) # Assign the multiplier of 15 to view the pages we want
        current_page = ''.join([final_site, '&start=', start_num])
        # Now that we can view the correct 10 job returns, start collecting the text samples from each

# Beautiful Soup HTML Extraction for each page:

        html_page = urllib.request.urlopen(current_page).read() # Get the page         
        page = BeautifulSoup(html_page) # Locate all of the job links
        jobs = page.find('td', attrs={'id': 'resultsCol'}).find_all('div', attrs={'class': "jobsearch-SerpJobCard"}, recursive=False)
        base_url = 'http://www.indeed.com'
        
# Scraping info from main job column and individual job page:

        for jobs in jobs:
            # Job Title:
            title_element = jobs.find('h2', attrs={'class': 'title'})
            title = title_element.a.text.strip()
            # Link to full job page:
            partial_job_link = title_element.find('a').get('href')
            link = base_url+partial_job_link
            print("The job link :", link)
            # Extracting details from full job page:
            job_page = urllib.request.urlopen(link).read()
            jobSoup = BeautifulSoup(job_page)
            try:
                details = jobSoup.find('div', attrs={'class': 'jobsearch-jobDescriptionText'}).text.strip()
            except AttributeError:
                details = "Unavailable"
            # Company:
            company = jobs.find('div', attrs={'class': 'sjcl'}).find('span', attrs={'class': 'company'}).text.strip()
            # Location:
            try:
                location = jobs.find('div', attrs={'class': 'sjcl'}).find('span', attrs={'class': 'location'}).text.strip()    
            except AttributeError:
                try: 
                    location = jobs.find('div', attrs={'class': 'sjcl'}).find('div', attrs={'class': 'location'}).text.strip()  
                except AttributeError:
                    location = "Unavailable"
            # Summary from job card:        
            summary = jobs.find('div', attrs={'class': 'summary'}).text.strip()
            # Date:
            date = jobs.find('div', attrs={'class': 'jobsearch-SerpJobCard-footer'}).find('span', attrs={'class': 'date'}).text.strip()
            # Adding data to CSV:
            writer.writerow([title.encode('utf-8'), company.encode('utf-8'), location.encode('utf-8'), summary.encode('utf-8'), link.encode('utf-8'), details.encode('utf-8'), date.encode('utf-8')])
            sleep(5)
    print("done writing files")
    file.close()
    return 


# End

In [132]:
seattle_info = skills_info(city = 'Seattle', state = 'WA')

Finale site : http://www.indeed.com/jobs?q=%22internship%22&l=Seattle%2C+WA
There were 810 jobs found, Seattle
Getting page 1
The job link : http://www.indeed.com/company/Institute-for-Systems-Biology/jobs/High-School-Intern-ce925db3aba6289a?fccid=46a724aeba6996aa&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=07f5cc4aded80ed5&fccid=734cb5a01ee60f80&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=d54c9212bc68db72&fccid=88e380af575ae18a&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=417a60c27d03c787&fccid=fe2d21eef233e94a&vjs=3
The job link : http://www.indeed.com/company/Institute-for-Systems-Biology/jobs/Summer-Undergraduate-Intern-de08dffe4541f24d?fccid=46a724aeba6996aa&vjs=3
The job link : http://www.indeed.com/company/Fred-Hutchinson-Cancer-Research-Center/jobs/Intern-Cancer-Biology-2715a1047037f84e?fccid=a513e92edbf529af&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=f2d155902d9920f9&fccid=b0157d60f7e532f6&vjs=3
The job link : http://www.indeed.com/company/D

The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C3x855-QZGygYX_tZSQl7YmWaq0lFRqI7kPasWKP5Qy4DHwpEj12DVccd7O0RBpBSUwp4Xw4wz71t4u19hRmJwZ09dFCLz0yR6j1gLMZfaPQFHK8OV8Z3DYMXApZGMnMDExYkTyF9d70sgqrTIc_CIkGGKXCSwr0z_b9CT_DJISfAk0FovT_lrUg0HqYQM5kcQtVZGKP4jBYVkhVTP8AtrBehdN50Ks6vsLF6_dYgcdHteAfBul0MkILwnBOslQgFGqZWFm8l9bnG9uTsT_wRdkNRgVlNOu2eQwQREvjb-aUFWJZkigFRjSQoXGVTuPr_hn1WNphxEcuHVTqj_wkGQ2O5mQiL_ltvSien1LocH3nmC3FEsRMMHg4j3L7rDClN8gFHe-Q9taeow5TEnIiBDNifBHeTeLXDIhyAcnItO9IOonbM5HCHA8p942cWUe6gNz4qKmecsdMLs7osKSi27e5Z5czRpN_w=
The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CsARmfH1XNQTa22oGIIJ18FtyAjbQsgfeQZpddTLaeHvZpIwnWfml1BZSIDOOc3g_oAraN52GNLI530UIEbaBjiPNixF_ordBcGeEZazsWjbbEZMZaF_tMAxRcKuJLPQPSZqcQGQvgS61SmaBvsQhB1N2YXc9caNnp8rmdF8GelUarin_P9W8rXHGe-f_d_51mzLyus8YyIylseJuFQhRz7jmIqEo-IXpmvSpJXNs0h_uFlPOfAK3EvaGv09XTLKsXSsjjo4sNMm7s8HCycx-vVG_TW6ApNaMbyAQ97Um_L7Z-j9Zg5XQvKNW6hb9EvS1bsxvcQA71IEEgaxtRFoFZuYsc6azjEJBzA-i7d_gd78HvlqxDHrHRrPkqT3coufz

The job link : http://www.indeed.com/rc/clk?jk=9f1d5a33173c3326&fccid=f183d3395dedda4d&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=b8be64d1f948bee3&fccid=b0157d60f7e532f6&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=a5568e152be726e2&fccid=729eb000fb24b6d7&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=0bc2d111d82bace5&fccid=2a80f36ff3f60bea&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=0f70639dfc37bcea&fccid=d124dbfd3785b597&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=9e44fd1dfffdbadb&fccid=c9e726792e354857&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=b66bca4e9174d3cd&fccid=fe2d21eef233e94a&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=7178260108d482e1&fccid=aef928e89977f7f0&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=2ae15872c4eac1f8&fccid=7f80fa2df8990e50&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=f28d606132e737cc&fccid=17d397c17f760b21&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=a25374506a1ef644&fccid=

The job link : http://www.indeed.com/company/Wilderness-Awareness-School/jobs/Coyote-Summer-Camp-Internship-cd6caff96102a3de?fccid=007258828a6f6515&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=ca33792afefbafb5&fccid=675c12bc7ffc102c&vjs=3
The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BU1oMTuMJSiKD6QSL_04fCHEWRxdwQuG3QkaxdZWD_wdFuQZ6ur8Ih1CmQSK2TTcjdIcrNCky9QUXp4VVemtx_oB2o0oLertovSS8GQoVH2yIjyoQuuWAlwjSEZ9nCxU7SPQ3EOtR1lwBy1wZf2HqHwVlcl00iC7iwt9ZF-jmHXZcazh3L0QeoLc3m9cGLzwy1TG-q3JPS8CODr__xtKunJKRV8cWu6QhFh0r1-5GVq-ka_GTQ-grqH9DXtExuHM2BQdt8LlkhJdszFAYi3Nbs7ivV6GEriHH-wICMErL67WGVNHz7wKyt5IrdBPwI7ZpbelcQQTkkdG8lJWoaxR6v5__Bl55z5zE7q8W8dk6syGlWjD3s-ScA5olN1DSSJiAafG9cisIhLjapD_8HIp8G3euqsjQRvAzIrXShLCdKsL_Ubx_XzdakkgGny9mJgwIpDyjoDQ0wWJRQLSnSDH4odBS_PtUA7Tzgb76ghEPmKw==&p=10&fvj=1&vjs=3
The job link : http://www.indeed.com/company/GAOTek/jobs/Event-Management-Virtual-Intern-or-Op-2134a83c3a98daaa?fccid=296fac4d90feb11e&vjs=3
The job link : http://www.indeed.c

The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0BKgzQyzTF1Q9mOsR1amaS-juVGLjHt5Cdom-gEF9y-xfJwY2lkC_-Z372KPgItZBFWIMsSheLWqO53sqiTHH9kfdVzk1i1vPqkc2qDS7WxK41Kk4JcsW4jnnZRMDAMDPWiaXi5DpZEJeXQ4cIXMUxLzdJtrIIv5PpBxhQHRWOcwYc_8UgMX4gSq21HhWxS4m-qGPlqiK_YXePldNuJh1iMdLKIxAus6bHcZHXLeeTI3UhAnuA3OnsTTFSKA58zBQcE2gXpScFbPJpA4w98QdXCeeMTfsuEY_Ij6H1HyLl5a6bl3aMH7SApu9bspwxA4vk-XJmKnUD2EqVHNz7VJqVb9dKj3sC2o_BjqEgnhb0gWDI-_R6Q0hTXTXvlozpFgGOHC75LBKbtGOIVGEVuykPoxbVHYuCwNqraFLmEGABNglYqNTgpZpgVkytQY8LgZp8Y4tKUrettERo63_yUUVCL4Y_-Oa6kT_dGezna9oEUkQ==&p=14&fvj=1&vjs=3
Getting page 15
The job link : http://www.indeed.com/company/The-Red-Barn-Farm/jobs/Animal-Care-Specialist-17dc9847522ccf06?fccid=f8f873bdaca10a72&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=7c107f43d744a16f&fccid=fe2d21eef233e94a&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=493f83fb64471ec9&fccid=a710372f724c1e2e&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=941369129732ced3&fccid=734cb5a01ee60

...

The job link : http://www.indeed.com/rc/clk?jk=62b8a054e146907e&fccid=734cb5a01ee60f80&vjs=3
The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0CsVqxrTdu7Y-QW9OzdKlKlQspxjF7alEdxbzpbFOvg9aN63o7WvwNLo14eTozuTXFEUhBXlOz_mLtrzXamoW_uYqSkqdxI6_U7X_uXhtXOWEsDyTpSbKmzTIevEFxkRx2rZA_f_c_NHUwIK3nkqGBEw8AyVzgS5XKBIyJdChqRmY40ACXSxrz-xX4RrsWshChZ8Yi6Cck_eDeu4Ie53HnHLP4PVHGIx-ABMhWCbRFWZS27ZtygeTEMofLCdGBIKGbFQ9zYJ2tHC67eio4dtWdWmO5rMJi_X_8U29mcQ-DdasfS-YNGmdmur9f8-7toT7VxmM_QAz-KZ0T8iVkth8QCwXlVJgwYu9LQ76n_tQ3opAyygNgL0qLo-p8uwvc0EngBPt3ZSar67aygN3-Yp-VDyoMIHD5LEmFPS9BKP21OTlqHpfiHTx_WXLIo8DOKZB4oy2FL5cYexhAiSLl67xx50RnvCS3wDqkpY4Ajwy6ULVUPNm80ogMEHVTUSk1R7eQBwaSCfylq07a4zCyOyewF6zRfBxQFqy7qyw0LnCFQrpUYVhzzZhAe09XtEXavkG3avFTMAJWpKDgx0wFod2zsrDQg-DvM7a72MmmH8ZytaquBDySws_lNlLgX-yq3SDq8-fn5GR7NJE6t9DJJ-QTNUHJXKc5M42otHCjkTw1QVZbbMXR6fwp5heb5YRQZcxR0jemzW3ILNPCAh8LH9z_rUdJAje1PxYdpV9jMG8CB90Qx6QyoF6ZJNtAoPTpR2EE7MmAQwHzj29mORWuIGnjB7zO8oFgLhxwyPZA-L8D43-_tgqO-_9fcayIlMblZNp1t1OeZoRa

The job link : http://www.indeed.com/rc/clk?jk=7b7f83738eef2b57&fccid=0739a410837ee13f&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=11f0a52a71c175bc&fccid=9af2e53838a29d1d&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=594ba6dd724b31ec&fccid=9af2e53838a29d1d&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=4d03459c496eed63&fccid=3c80b0f0143c1c2e&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=a6d2dde1233b1910&fccid=fe2d21eef233e94a&vjs=3
The job link : http://www.indeed.com/company/Trans-Lines,-Inc/jobs/CDL-Class-Otr-Driver-892754d41af8a48a?fccid=1345e9880dd3573f&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=b8c130af1374e43b&fccid=734cb5a01ee60f80&vjs=3
The job link : http://www.indeed.com/rc/clk?jk=62b8a054e146907e&fccid=734cb5a01ee60f80&vjs=3
The job link : http://www.indeed.com/company/American-Employment-Institute/jobs/Web-Developer-2e806f6bbfcea4fc?fccid=7b845a1fca7beafa&vjs=3
Getting page 51
The job link : http://www.indeed.com/rc/clk?jk=b8c130af1374

The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_1G4MOf4l_p8Xwb6gxqXskJGh74Fij0qpfj-k8BoCzE9__EdyHx-GNQzOYftD-mmHjeXINmXzub_RYWIcwAFSzSKnNgFpK3kfKzWMlNozP7JQZ32VbhsC1REzD37zScyzzT4OZmGjd-McMtR6-MTlbwR2XoEczXnE51txCTGwp3q-WdJOvt3YyYHYiNv4nFvEoyiOWCQdIfQC2zU0pqADwYGmzUzl5jUUnVmwVb2LmmNLFcDVFLsD_stBvxa3UHnYQZnk0EaI8jrI4Al1hI-f8vupoPgmkA1g5iVdJiBak4YxJN-lgSFcb-W8--udfMjS73hDpbQtcjtK5v2gB7HCHpSt1Prv82gGDyL75X1yeg9YkropAnIKML-l_zQtssSkasoonQc_1qy3AvCJYkXY7u6SQVcCkweGPA6YBI7-vE0H2Rh5tmn75X4i1TAVj1t7L1pBrAW4DWWUuFZzE1W8v0_QKnNcJMqlhY1k9M8cNwr9yMrkNFLRXPUnWkmqjC49Ndy2F1WeIOJb4Hq7vJpc5hJFl03gRpAXOYg0cdIWKsu1ZNV0FGrIyGOeCmwwiApNrF9LGyp_8XNYYuDzi6xf6ZhYPOp6RFwZ_BGXd1FgkuDkiO1YaNUI7e90M6triMTpbJlr9T1O90syctt_FUpYv_05GHm5AVPEh4473XgCyNrxrNVlFP7ldoEfAAkdIVpQllHN1U8hVt49oRGYlfUf65j9FJG5h1y9wzVFfnDJbD4N7quFWEHzs6UCTqpDDHjApUnz9zoOzV6RLOTNyR2wAmm09NZAin7UMjc8hOykgV3l9PBho9m4qvICwQefeli3F_OMEF3hgdsRLAdkd1wxtw1-Udglg-kTcgT-lJVmgM_KswHBeXJnpeu2kWRDE7B-SDaYtD869p3HH0Q41GAVQiJGfhfecXG67kQA

The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_1G4MOf4l_p8Xwb6gxqXskJGh74Fij0qpfj-k8BoCzE9__EdyHx-GNQzOYftD-mmHjeXINmXzuT_Su-131tw0gZVtMM2kjRZAtOffnbGHMBIK8udau1R6No7yBVdm2I-4gNDlsdFoc2xmmqWqOmx_dqe1xPANEJS4thDp-gVBNG0_lEnwAooIgeSY1ANr3MTHLoq44oEVOGahRBBgHr6sLK9aQP4lRpyu4XJcLzTAW8GMLuZLT-zkxSI-h8Y7X1VM__xLiYPBIMxz2awVuFsuNDOqw7i96pisaJ9FPHxdmQqiVjbHuvYFBt3WGJzX0y5X4ynN1eaZ6rDN8LxE0uNuQxDiffiU8QBdZXQuQFQApTHn7cYoyraKlGxblzBetS7IjCKbGbgN7YWTxfx9kv7k4feYGQ12yBdQMm6QyrjQRl0Gd7k8629muezdaZK3vlSSuS2yBtveHJq5JCX94du4oyaQdL79j3TlTcOvpO5p-CdM-lQtUkxIbfeOMqP7Jm2R_gTSZCDrC8uqURKEpJS0sELnv7QauHxi8NQC4MHTcJaLepjqLLnR6r3IqSZ1d-XdCfzI_JGdvib2oYcoDb-coMCagKzadQzGvWobSnyJvbti8esc8YD96b4bbTaQ50JYZfthqowiZJy93mwhXnvkZL_5XThnqu6C_woInF284nCauMh1vM55y6H_F-Djo3Txam4Nd1OMSIfT43P0nknSSgPw7TZDb_oUoT5r4MbVe3bBM9wKLBjgRX3eQnuKEG4EzkKO7gCH2GZdL1xiyXD3VgZWIuNOY8p0A5GE2ayuXf0BEiNPdzDpvpjGS8oU4aLkfpSibNn0BqjvqldZk9nuVrT227BbmmIorDohcTKSBw950HTVkT_EFk0_1qV7UFAJMZRC_sMI2O8ILQJfM8og78s7PR2lCbT3wMRh

The job link : http://www.indeed.com/pagead/clk?mo=r&ad=-6NYlbfkN0C_1G4MOf4l_p8Xwb6gxqXskJGh74Fij0qpfj-k8BoCzE9__EdyHx-GNQzOYftD-mlp9Kevmlhs0KAolCPrGj951Pux4qUusffWKuLKpVJnBZR2-5LwO0Hmx6sGroZEomY6jxk5aDb9jMv0v6uTmJPv1lNjlODc5-3FFmjz4Mmc_YK1Fc2gC_8uUVL8pakKjbi8CACNbIVvLrnSMk_rdgfYKLL5G_-VHRftcJwP6EfMiQIsPZ9L6XTGup0XU_Z85Dw4s8-LXETpVNzef_DNKGIy-oRs68kSX1RMbAdIX7RiaLuD2qwL-2i6FpEvTn6C4FrzcxFhL_TrC0mwnKNEvUCpx0CYVOrI_MGbksj6NI0rIwGtplaG_izYfydXSkZS90bKJLJUT0_p0BQYwpE-yK-KeI3Nr5YZoMLK0W0bOi_dhjOfTSrV5tJ9n8qmpORTh17B4vd1PNxO7idOwwGmqhk-90tW40oM-odT0IhVw_F2Oj_8yf8SQzb5zd-tKmAW4lUiNwVf_qHzdpP2HFyJna1u2hVOFhr5GJz09k_5nGKr6OnuNQIs2TstJ0FR2R_AkBSuXXfYVj777PGMwl2KBm12C0KT8PHcmC6PAGz_scCkR4_4ssrx-svBRxoXudcZ4SnleMFKxvwM68_3rCNVd4oTIgHhTWYxGwGbGFNjdz8XhhR73EO6VwJd--eWEZIWZq3K1EVW0pFIyCBTgqCjgsHeF3z-MtIu3L0Z_d2cAmLt7vxtIen2NrCF7X1Asoek7JjkftGRov-CFYjLyPiv8VpkOJm_t0w7ZKM7_liP4TqtGm-OloAAdw7YQkjamNEC0qFf3_-cVrg6I7tOA8JkAZHk1eK-fFc6AUHHFH7GjCwzLD58qzqNo7UGyTjtzv3tviMAfBX27m53nQLNHrwHJPy1jJOy4VwYCdZbM4AH